In [190]:
import pandas as pd
import numpy as np

In [207]:
bus_stages = [
    'T.NAGAR', 'SAIDAPET', 'ANNA UNIV', 'WPTC', 'SRP TOOLS',
    'KANDANCHAV', 'THORAIPAKKAM', 'M K CHAVADI', 'KARAPAKKAM',
    'SHOLINGANALLUR', 'KUMARAN NG', 'CHEMMANCHE', 'NAVALUR',
    'SIPCOT', 'CHURCH', 'PAL. CHEMI', 'HINDUSTAN', 'KELAMBAKKAM',
    'KOMAN NAGAR', 'ENGG', 'CHENGAMMAL', 'KALAVAKKAM','THIRUPORUR'
]

boarding_counts = {}

from_stage=input("Enter From Stage: ").upper()
from_stage_index = bus_stages.index(from_stage)

df=pd.read_csv('times.csv')
arr=df[from_stage]
lists=arr.tolist()

to_stages = bus_stages[:]

for stage_index in range(len(to_stages)):
    stage = to_stages[stage_index] 
    df=pd.read_csv('519-up-buses.csv')
    df= df[(df['From Stage'] == from_stage) & (df['To Stage'] == stage)]

    df['Ticket Issued Time'] = pd.to_datetime(df['Ticket Issued Time'], format='%H:%M:%S', errors='coerce')
    df['Hour'] = df['Ticket Issued Time'].dt.hour

    df = df.groupby('Hour')['Schedule Name'].count().reset_index()
    df.columns = ['Hour', 'Bus Count']
    hours = pd.DataFrame({'Hour': range(24)})
    df = hours.merge(df, on='Hour', how='left').fillna(0)

    df['Bus Count'] = df['Bus Count'].astype(int)
    df['Cumulative Boarding'] = df['Bus Count'].cumsum()
    df['Minutes'] = df['Hour'] * 60

    def estimated_boarding(target_time):
        target_minutes = pd.to_datetime(target_time).hour * 60 + pd.to_datetime(target_time).minute

        previous_index = df[df['Minutes'] <= target_minutes].index[-1]
        next_index = previous_index + 1

        previous_minutes = df.loc[previous_index, 'Minutes']
        next_minutes = df.loc[next_index, 'Minutes']
        previous_boarding = df.loc[previous_index, 'Cumulative Boarding']
        next_boarding = df.loc[next_index, 'Cumulative Boarding']

        ratio = (target_minutes - previous_minutes) / (next_minutes - previous_minutes)
        estimated_boarding = previous_boarding + (next_boarding - previous_boarding) * ratio
        return estimated_boarding
    
    estimated_counts = {}
    passenger_boarding = []

    for time in lists:  
        estimated_count = estimated_boarding(time)
        estimated_counts[time] = estimated_count.round(0)

    for time, count in estimated_counts.items():
        passenger_boarding.append(count)

    arr = [passenger_boarding[0]]
    for i in range(1, len(passenger_boarding)):
        difference = passenger_boarding[i] - passenger_boarding[i - 1]
        arr.append(difference)
        boarding_counts[stage] = arr
        
boarding_df = pd.DataFrame(boarding_counts)
boarding_df.insert(0, f"{from_stage} STOP", lists)
boarding_df.to_excel(f"{from_stage} STOP.xlsx")
boarding_df


Enter From Stage: saidapet


SAIDAPET STOP  T.NAGAR  SAIDAPET  ANNA UNIV  WPTC  SRP TOOLS  KANDANCHAV  \
0       05:07:00      0.0       0.0        2.0   2.0        2.0         7.0   
1       05:27:00      0.0       0.0        0.0   1.0        2.0         0.0   
2       05:47:00      0.0       0.0        0.0   1.0        2.0         0.0   
3       06:07:00      0.0       0.0        0.0   0.0        2.0         0.0   
4       06:27:00      0.0       0.0        0.0   0.0        1.0         2.0   
5       06:47:00      0.0       0.0        1.0   1.0        2.0         1.0   
6       07:11:00      0.0       0.0        0.0   1.0        2.0         3.0   
7       07:21:00      0.0       0.0        0.0   1.0        1.0         1.0   
8       07:31:00      0.0       0.0        1.0   1.0        2.0         2.0   
9       07:41:00      0.0       0.0        0.0   0.0        1.0         1.0   
10      07:51:00      0.0       0.0        0.0   1.0        1.0         2.0   
11      08:01:00      0.0       0.0        0.0   1.0        1.0         1.0   
12      08:11:00      0.0       0.0        1.0   1.0        2.0         1.0   
13      08:21:00      0.0       0.0        0.0   1.0        1.0         1.0   
14      08:31:00      0.0       0.0        1.0   1.0        2.0         1.0   
15      08:41:00      0.0       0.0        0.0   0.0        1.0         0.0   
16      08:51:00      0.0       0.0        1.0   1.0        2.0         1.0   
17      09:01:00      0.0       0.0        0.0   1.0        1.0         1.0   
18      09:11:00      0.0       0.0        0.0   0.0        3.0         2.0   
19      09:21:00      0.0       0.0        0.0   0.0        3.0         1.0   
20      09:31:00      0.0       0.0        0.0   0.0        2.0         2.0   
21      09:41:00      0.0       0.0        0.0   0.0        3.0         1.0   
22      09:51:00      0.0       0.0        0.0   0.0        3.0         2.0   
23      10:01:00      0.0       0.0        0.0   0.0        2.0         1.0   
24      10:11:00      0.0       0.0        0.0   0.0        1.0         1.0   
25      10:31:00      0.0       0.0        1.0   0.0        1.0         2.0   
26      10:51:00      0.0       0.0        0.0   0.0        1.0         1.0   
27      11:08:00      0.0       0.0        0.0   0.0        1.0         1.0   
28      11:28:00      0.0       0.0        1.0   0.0        1.0         0.0   
29      11:48:00      0.0       0.0        1.0   0.0        1.0         0.0   
30      12:08:00      0.0       0.0        0.0   1.0        3.0         1.0   
31      12:28:00      0.0       0.0        0.0   2.0        4.0         2.0   
32      12:48:00      0.0       0.0        1.0   3.0        4.0         3.0   
33      13:08:00      0.0       0.0        0.0   1.0        2.0         1.0   
34      13:28:00      0.0       0.0        0.0   0.0        1.0         1.0   
35      13:48:00      0.0       0.0        1.0   0.0        1.0         1.0   
36      14:08:00      0.0       0.0        0.0   0.0        0.0         0.0   
37      14:28:00      0.0       0.0        0.0   0.0        1.0         0.0   
38      14:48:00      0.0       0.0        0.0   0.0        1.0         0.0   
39      15:08:00      0.0       0.0        0.0   0.0        1.0         0.0   
40      15:28:00      0.0       0.0        0.0   0.0        1.0         0.0   
41      15:48:00      0.0       0.0        0.0   0.0        1.0         1.0   
42      16:11:00      0.0       0.0        0.0   0.0        1.0         1.0   
43      16:21:00      0.0       0.0        0.0   1.0        1.0         0.0   
44      16:31:00      0.0       0.0        0.0   0.0        1.0         1.0   
45      16:41:00      0.0       0.0        0.0   0.0        0.0         0.0   
46      16:51:00      0.0       0.0        0.0   1.0        1.0         1.0   
47      17:01:00      0.0       0.0        0.0   0.0        1.0         0.0   

    THORAIPAKKAM  M K CHAVADI  KARAPAKKAM  ...  SIPCOT  CHURCH  PAL. CHEMI  \
0            0.0          3.0         4.0  ...     0.